### Description:

AES-128 algorithm with a hardcoded key is available as service.

To perform encryption, the binary requires 16-bytes plaintext (in hex):

```
    ../wbcenc --ptext="010203040506070809000a0b0c0d0e1f"
```    
The result is printed to console:
```
    97935516ADA083B7A1F13BD9D89EBC11 0803050A0502040107030600030504...30506060B0A060205
```    
where:
```
    97935516ADA083B7A1F13BD9D89EBC11 is a ciphertext
    
    0803050A0502040107030600030504...30506060B0A060205 is a trace (information left by a programmer)
```
### Tips:

A 'trace' is a side-channel information left by a programmer to debug the implementation.

In this example a trace consists of a random data plus at certain position a trace contains Hamming weights of bytes for a 10th round input state of the AES-128 algorithm, i.e., a State before the Sbox operation in the last round. Those Hamming weights contain also some noise (they are not perfectly measured).

### Task:

Your task is to find the Master key (round key 0) embedded into the binary. 

The master key is in the form of HEIG{XXXXXXXXXX}, where X is an ASCII printable symbol.

### Leakage illustration

<img src="support/Slide.png">

In [1]:
import numpy as np
import binascii
import string
import re
import socket

HOST = 'iict-mv330-sfa'
PORT = 4003

import sca_training
#----------------------------------------------------------------------------
# This function calls tested binary ../wbcenc either with a user-defined 
# plaintext (if plaintext satisfies all the requirements) or with a 
# predefined plaintext (the same as in the header above)
#
# INPUTS:
#     plaintext - a string of 32 symbols representing 16 hex bytes of ciphertext
#     verbose   - a flag to print values in the function call or not
# OUTPUTS:
#     output  - a raw binary output
#     ctext   - resulted ciphertext converted to numpy array of uint8
#     trace   - a trace associated with encryption process
#----------------------------------------------------------------------------
def binary_aes128_encrypt(plaintext, verbose=False):
    import subprocess
    pattern = '[0-9A-F]{32,324}'
    
    #Check the correctness of the plaintext
    if all(c in string.hexdigits for c in plaintext) and len(plaintext) == 32:
        
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.connect((HOST, PORT))
            s.recv(1024)
            s.sendall(plaintext.encode())
            s.sendall(b'\n')

            output = s.recv(1024)
    
    #Get the ciphertext (first 32 symbols) and the trace (last 32 symbols)
    result = re.findall(pattern, str(output))
    #print(result[0], result[1])
    
    #Transform the result into numpy array
    ctext = np.frombuffer(binascii.unhexlify(result[0]), dtype=np.uint8)
    trace = np.frombuffer(binascii.unhexlify(result[1]), dtype=np.uint8)

    if (verbose):
        print('Binary output:', output.strip())
        print('Ciphertext as numpy array:', ctext)
        print('Trace as numpy array:', trace)
    
    return output, ctext, trace


#----------------------------------------------------------------------------
# This function calls supportive library to compute a master key from
# the last round key.
#
# INPUTS:
#     last_round_key - a numpy array of 16 elements representing 16 hex bytes of
#                      the last round key, i.e., Round Key 10
# OUTPUTS:
#     key_schedule  - 11 round keys of 16 bytes each (key_schedule[0,0,:] is the
#                     master key
#----------------------------------------------------------------------------
# key_schedule = sca_training.inverse_key_expansion(last_round_key)

In [2]:
output, ctext, trace = binary_aes128_encrypt("010203040506070809000a0b0c0d0e1f", verbose=True)

Binary output: b'97935516ADA083B7A1F13BD9D89EBC11 0303050108030606060204000302050805050A0509020201010A05060306070401080607090609060603040506060505030908060B080506080308030205040505080608050707020505050A0305060406020403080405040A090B05020703060603030A'
Ciphertext as numpy array: [151 147  85  22 173 160 131 183 161 241  59 217 216 158 188  17]
Trace as numpy array: [ 3  3  5  1  8  3  6  6  6  2  4  0  3  2  5  8  5  5 10  5  9  2  2  1
  1 10  5  6  3  6  7  4  1  8  6  7  9  6  9  6  6  3  4  5  6  6  5  5
  3  9  8  6 11  8  5  6  8  3  8  3  2  5  4  5  5  8  6  8  5  7  7  2
  5  5  5 10  3  5  6  4  6  2  4  3  8  4  5  4 10  9 11  5  2  7  3  6
  6  3  3 10]


### ATTACK CODE
Here you need to implement your attack which gives you the last round key.

Once the last round key is found - you need to compute the master key: this can be done with the function get_master_key()

### Programming tips
#### Numpy code
* np.arange(256).astype(np.uint8)
* np.bitwise_xor(m1, m2)
* np.where()

#### Code prepared for training
* sca_training.invSbox[sbox_out]
* sca_training.shift_rows(trace)

#### Various cod
* ''.join('{:02x}'.format(c) for c in int_array)